In [9]:
import pandas as pd
import hashlib
from thefuzz import fuzz
from postal.expand import expand_address

pd.set_option("display.max_colwidth", None)

In [10]:
q2y = pd.read_excel("out_q2y_2016.xlsx")

In [11]:
expenses = pd.read_excel("train_data/Expenses/parsed/2016.xlsx")

In [21]:
# Получение хеша
def get_hash(str):
    return hashlib.md5(str.encode())

In [ ]:
# Форматизация адреса q2y
for ind in range(q2y.shape[0]):
    q2y.replace(q2y.iloc[ind, 1], expand_address(q2y.iloc[ind, 1])[0], inplace=True)

# Форматизация адреса expenses
for ind in range(expenses.shape[0]):
    expenses.replace(
        expenses.iloc[ind, 7], expand_address(expenses.iloc[ind, 7])[0], inplace=True
    )

In [23]:
# Создание таблицы индексов
uhash_id = []

for ind in range(q2y.shape[0]):
    uhash_id.append(get_hash(q2y.iloc[ind, 1]).hexdigest())

uhash_id = pd.DataFrame(uhash_id)

id_df = pd.concat([uhash_id, q2y.iloc[:, 1]], axis=1)

In [ ]:
baseline = 89
expenses_stack = []

for ind in range(id_df.shape[0]):
    if fuzz.WRatio(id_df[ind, 1], expenses.iloc[ind, 7]) >= baseline:
        expenses_stack.append(expenses.iloc[ind, 7])

expenses_stack = pd.DataFrame(expenses_stack)

id_df = pd.concat([id_df, expenses_stack])
id_df.columns = ["uhash_id", "address_q2y", "address_expen"]

In [25]:
id_df

,uhash_id,address_q2y
0,13c7e2183056327b0e518647b1a58be3,помещение объект номер 2 вспомогательныи 675000 амурская область г благовещенск святителя иннокентия переулок деревня 17
1,a25f47435fe826d04005bd616af282fd,административное здание литера а а1 676980 константиновскии раион север константиновка константиновская улица деревня 40
2,9508898a4efa48e4492de9cc2c6d72ab,административное здание 676246 амурская область г зея жуковскии переулок деревня 15
3,171dac669a1f86a94e21a85d4d6bbfe1,административное здание 676722 амурская область буреискии раион поселок городского типа новобуреискии советская улица деревня 19
4,a5fe66ee4645dd175f4df61af518ae79,административное здание 676776 амурская област г раичихинск победы улица деревня 49
...,...,...
523,ba656c4a69432b131dc29759263dfda7,пристроика корпус тп номер 2 689000 чукотскии автономныи округ г анадырь улица рультытегина тп-2
524,b1c82aeff5ba811edb80f311f27cb283,земельныи участок 689251 чукотскии автономныи округ провиденскии раион поселок городского типа провидения улица набережная-дежнева деревня 18
525,424c84192565e00f54ab7d9d24ca52c6,земельныи участок 689202 чукотскии автономныи округ раион иультинскии гора сопка
526,5a09837e3f709190152e0e5528b535d7,здание ркц 689450 чукотскии автономныи округ г билибино улица ленина 6а комп


In [ ]:
# Объединение датафреймов
q2y_expen_frame = id_df.merge(q2y, on="address_q2y", right_index=False).merge(
    expenses, on="address_expen", right_index=False
)
q2y_expen_frame.shape